In [417]:
import pandas as pd
import numpy as np
import os
import datetime
from scipy.sparse import csr_matrix


In [418]:
## Get address of where the repository is located on your computer.

# Will raise exception is you're in the wrong folder.
if not os.getcwd().endswith("02443-SimSolarEnergy\\code"):
    raise Exception("Change working directory to the code folder.")
repo_loc = os.getcwd()[ : -len("\\code")]


In [419]:
# Load data
# Use 16 instead w/ short whatdoing codes
activ = pd.read_csv(repo_loc + "\\data\\UK-TUS-15-0616\\activity.csv")
house = pd.read_csv(repo_loc + "\\data\\UK-TUS-15-0616\\household.csv")
indiv = pd.read_csv(repo_loc + "\\data\\UK-TUS-15-0616\\individual.csv")

for data in [activ, house, indiv]:
    print(data.head())
    print("\n\n")

     serial  pnum               t_start  eptime  whatdoing_exact  WhereWhen  \
0  11011202     1  2014-12-11T03:00:00Z     110              110         11   
1  11011202     1  2014-12-11T04:50:00Z      10             8219         11   
2  11011202     1  2014-12-11T05:00:00Z      10              310         11   
3  11011202     1  2014-12-11T05:10:00Z      10             3210         11   
4  11011202     1  2014-12-11T05:20:00Z      10             3110         11   

   whatdoing  
0          0  
1          8  
2          0  
3          3  
4          3  



     serial  num_adult  num_child  num_room  microwav  dishwash  dhhtype
0  11010903          2          0         8      True      True      3.0
1  11010904          2          0         6     False      True      3.0
2  11010906          3          0         4      True     False      7.0
3  11010907          2          1         3      True     False      2.0
4  11010908          1          0         6      True      True    

In [420]:
# Make dataset smaller
np.random.seed(0)
rows_to_take = np.random.randint(0, activ.shape[0], 10**5)
activ = activ.iloc[rows_to_take, :]
activ

,serial,pnum,t_start,eptime,whatdoing_exact,WhereWhen,whatdoing
305711,16171115,1,2014-11-20T16:00:00Z,60,2120,11,2
435829,18280401,1,2014-06-01T03:50:00Z,80,110,11,0
117952,12290909,1,2014-10-30T21:10:00Z,10,111,11,0
152315,13221201,2,2015-01-10T18:20:00Z,40,8210,11,8
359783,17140714,2,2014-09-03T00:20:00Z,40,110,11,0
...,...,...,...,...,...,...,...
397451,18050111,1,2015-02-22T09:10:00Z,10,9950,11,9
38326,11180819,2,2014-09-03T06:30:00Z,40,8110,11,8
385223,17290919,2,2014-10-11T07:40:00Z,30,300,11,0
453101,19080603,2,2015-06-20T13:40:00Z,20,5310,11,5


In [421]:
# Check for nulls
activ.isna().any()

serial             False
pnum               False
t_start             True
eptime             False
whatdoing_exact    False
WhereWhen          False
whatdoing          False
dtype: bool

In [422]:
# What proportion of time values are null?
len(activ[activ.t_start.isna()])/len(activ)

0.00358

In [423]:
# Not many out of the total, so let's just delete them
activ = activ[activ.t_start.isna() == False]

In [424]:
# Only take from one year
activ = activ[activ.t_start.str.startswith("2015")]

In [425]:
# First of all, only take the 11's
activ = activ[activ.WhereWhen == 11]

# Also let's look at the time preiod
np.sort(activ.eptime.unique())

array([  10,   20,   30,   40,   50,   60,   70,   80,   90,  100,  110,
        120,  130,  140,  150,  160,  170,  180,  190,  200,  210,  220,
        230,  240,  250,  260,  270,  280,  290,  300,  310,  320,  330,
        340,  350,  360,  370,  380,  390,  400,  410,  420,  430,  440,
        450,  460,  470,  480,  490,  500,  510,  540,  580,  600,  650,
        660, 1160, 1430], dtype=int64)

In [426]:
# Add unique person identifier:
activ["person_id"] = activ["serial"] + activ["pnum"]/100

In [427]:
activ.head()

,serial,pnum,t_start,eptime,whatdoing_exact,WhereWhen,whatdoing,person_id
152315,13221201,2,2015-01-10T18:20:00Z,40,8210,11,8,13221201.02
122579,13020219,1,2015-02-15T18:40:00Z,20,8310,11,8,13020219.01
448242,19050115,1,2015-04-17T05:00:00Z,10,8310,11,8,19050115.01
374564,17221202,1,2015-01-11T13:10:00Z,10,3110,11,3,17221202.01
170584,14020318,2,2015-03-22T11:30:00Z,10,3130,11,3,14020318.02


In [428]:
# Now change time into time "states"
# First cahnge to datetime
time_temp = pd.to_datetime(activ.t_start)

# Now get days per month (in order)
days_in_month = np.array([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31])

# Now calculate the number of days for each entry
month_days = np.array([sum(days_in_month[:(i - 1)]) for i in time_temp.dt.month]) * 24 * 6
days = (time_temp.dt.day - 1) * 24 * 6

# ... and hours and minutes
hours = time_temp.dt.hour * 6
minutes = round(time_temp.dt.minute/10)

# Now add to activ
activ["time_period"] = month_days + days + hours + minutes
activ.time_period = activ.time_period.astype(int)

In [429]:
# Check: we know the time of the last value as 21:40 on decemeber 12th (right hand side).
# We can calculate what the period should be (left hand side)
activ.sort_values("t_start").iloc[-1, :].time_period == (365-31+11)*24*6 + 21*6 + 4

False

In [430]:
# Change to numpy for rest (it's faster). Save index, row_names.
activ_columns = {}
activ_ind = activ.index
for i in range(activ.shape[1]):
    activ_columns[activ.columns[i]] = i
activ_array = np.array(activ)

array([[13221201, 2, '2015-01-10T18:20:00Z', 40, 8210, 11, 8,
        13221201.02, 1406],
       [13020219, 1, '2015-02-15T18:40:00Z', 20, 8310, 11, 8,
        13020219.01, 6592]], dtype=object)

In [431]:
# Change to true to debug (and don't use the whole index
# in the loop!)
test = False

# This will be the list of rows we'd like to add
new_rows = []

# Loop through all activity entries
for i in range(activ_array.shape[0]):
    
    # Save variables for this row
    row = activ_array[i, :]
    duration = row[activ_columns["eptime"]]
    if duration < 15: # <- if duration is only 10 min, we don't need to add anything
        continue
    start_time = row[activ_columns["time_period"]]
    
    # Add new rows based on the number of periods
    for time_add in range(10, duration, 10):        
        new = row.copy()
        period = int(start_time + time_add/10)
        max_period = 365*24*6
        if period > max_period - 1:
            period = period % max_period
        new[activ_columns["time_period"]] = period
        new_rows.append(new)
        
        # For debugging
        if test:
            print("Time add =", time_add)
    
    # For debugging
    if test:
        print("Start time = %0.0f, Periods = %0.0f" % (start_time, duration))
        print(new_rows)


In [432]:
# Check to see if we have added to correct number of rows
t = (np.array(activ.eptime[activ.eptime > 10]) - 10)/10
sum(t) == len(new_rows)

True

In [433]:
# We have! Good, so now we'll vstack the rows
activ_array = np.vstack([new_rows, activ_array])

In [439]:
# Good, now let's find what people were up to in the next time period
next_act = np.empty(activ_array.shape[0], dtype=int)

# For debugging
test = False

# Create sparse. This will speed up the loop in the next cell.
# Note we leave out column w/ index 2, since these are the dates (strings)
# which cannot be put into the sparse matrix
activ_csr = csr_matrix(activ_array[:, [0, 1, 3, 4, 5, 6, 7, 8]].astype(float))
N = activ_csr.shape[0]

# Get indeces (-1 because we took out the dates)
person = activ_columns["person_id"] - 1
time = activ_columns["time_period"] - 1
doing = activ_columns["whatdoing"] - 1

In [440]:
# Loop through all entries
for i in range(activ_csr.shape[0]):
    
    # Get details of row
    person_id = activ_csr[i, person]
    tp = activ_csr[i, time]
    next_tp = tp + 1
    
    # Look for entries that are with the same person and next time period.
    # The .nonzero() is because sparse matrices can't handle boolean indexing,
    # so the .nonzero() takes the indeces of the rows which return True.
    person_acts = activ_csr[(activ_csr[:, person] == person_id).nonzero()[0], :]
    next_acts = person_acts[(person_acts[:, time] == next_tp).nonzero()[0], :]
    
    # Save the activity in the next time period
    if next_acts.shape[0] != 1: # <- if there's no or multiple next activities
        next_act[i] = -1 
    else:
        next_act[i] = next_acts.toarray()[0][doing] # <- weird 0 index has to do with how sparse are converted to arrays

    # Print progress
    if i % (10**3 * 5) == 0:
        print("At row = %0.0f" % i)
    
    # Debug
    if test:
        print(activ_csr[i, :])
        print("---")
        print(next_acts)
        print(next_act[i])
        print("--------")

At row = 0
At row = 5000
At row = 10000
At row = 15000
At row = 20000
At row = 25000
At row = 30000
At row = 35000
At row = 40000
At row = 45000
At row = 50000
At row = 55000
At row = 60000
At row = 65000
At row = 70000
At row = 75000
At row = 80000
At row = 85000
At row = 90000
At row = 95000
At row = 100000
At row = 105000
At row = 110000
At row = 115000
At row = 120000


In [445]:
# Check the next_act values:
print(len(activ_array) == len(next_act))
print(np.unique(next_act))

True
[-1  0  1  2  3  4  5  6  7  8  9]


In [448]:
# Looks good! We'll add next_act to the array...
activ_array = np.hstack([activ_array, next_act.reshape(len(next_act), 1)])

In [453]:
# ...and update the column labels
activ_columns["next_whatdoing"] = 9

In [457]:
# Good, so now we can start making the transition matrices.
# Note, tp stands for time period

# Init
total_tp = 144
P_t = np.zeros([total_tp, 10, 10]) # <- 3D array for 144 * transition matrices

# For debugging
test = False

# Loop through all activity entries again
N = activ_array.shape[0]
for i in range(N):
    t = activ_array[i, activ_columns["time_period"]] % 144 # <- % 144 to take all days in the years
    j = activ_array[i, activ_columns["whatdoing"]] # <- current state
    k = activ_array[i, activ_columns["next_whatdoing"]] # <- next state
    if k == -1:
        continue
    P_t[t, j, k] += 1 # <- add one to probability, we'll divide later
    if test:
        print(activ_array[i, :])
        print("At time %0.0f, the person is transitioning from %0.0f to %0.0f" % (t, j, k))
        print("------")

In [500]:
# Normalize. There is a warning that comes out here, but its ok!
for t in range(144):
    P_t[t] = (P_t[t].T/np.sum(P_t[t], axis=1)).T # <- transpose and transpose back to match axes with division operator
    for row in range(10):
        if np.isnan(P_t[t, row]).any(): # <- replace nans with 0s
            P_t[t, row] = np.zeros(10)

C:\Users\Jan Warhuus\.virtualenvs\02443-SimSolarEnergy-Fn5Vr-Iv\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [502]:
# Test for nans
np.isnan(P_t).any()

False

In [503]:
# :D
# So, P_t is a 3D matrix of 144 transition matrices where index
# 0 is the time period (0 to 143), index 1 is the current state (0 to 9),
# and index 2 is the next state (0 to 9).

# Let's save everything:
np.save(repo_loc + "\\data\\transition_matrices.npy", P_t)

NameError: name 'repoloc' is not defined